In [3]:
from __future__ import print_function
import numpy as np 
import pandas as pd 

class TreeNode(object):
    def __init__(self, ids=None, children=None, entropy=0, depth=0):
        self.ids = ids if ids is not None else []
        self.entropy = entropy
        self.depth = depth
        self.split_attribute = None
        self.children = children if children is not None else []
        self.order = None
        self.label = None
    
    def set_properties(self, split_attribute, order):
        self.split_attribute = split_attribute
        self.order = order
    
    def set_label(self, label):
        self.label = label

def entropy(freq):
    freq_0 = freq[np.array(freq).nonzero()[0]]
    prob_0 = freq_0 / float(freq_0.sum())
    return -np.sum(prob_0 * np.log2(prob_0))

class DecisionTreeID3(object):
    def __init__(self, max_depth=10, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
    
    def fit(self, data, target):
        self.data = data
        self.target = target
        self.labels = target.unique()
        self.features = list(data.columns)
        self.Ntrain = data.shape[0]
        
        ids = range(self.Ntrain)
        self.root = TreeNode(ids=ids, entropy=self._entropy(ids), depth=0)
        #Chưa biết phần tử nào là root gốc nên thiết lập tất cả thuộc tính là root ,depth=0
        queue = [self.root]
        
        while queue:
            node = queue.pop()
            if node.depth < self.max_depth and len(node.ids) >= self.min_samples_split:
                node.children = self._split(node)
                if not node.children:
                #Nếu không phải node lá thì gán label và duyệt lại vòng lặp
                    self._set_label(node)
                queue.extend(node.children)
            else:
                self._set_label(node)
    
    def _entropy(self, ids):
        #Nếu duyệt hết các phần tử còn lại=0 thì return 0
        if len(ids) == 0:
            return 0
        freq = np.array(self.target.iloc[ids].value_counts())
        return entropy(freq)
    
    def _split(self, node):
        #Tính các node con, tìm trong node con entrophy tốt nhất thì duyệt theo node đấy
        ids = node.ids
        #ids là địa chỉ danh sách các node dự tính
        best_gain = 0
        #Split là mảng các thuộc tính tốt nhất để tính entrophy
        best_splits = []
        
        best_attribute = None
        order = None
        sub_data = self.data.iloc[ids, :]
        for i, att in enumerate(self.features):
            values = self.data.iloc[ids][att].unique().tolist()
            if len(values) == 1:
                continue
            splits = []
            for val in values:
                sub_ids = sub_data.index[sub_data[att] == val].tolist()
                splits.append([sub_id for sub_id in sub_ids])
            # don't split if a node has too small number of points
            if min(map(len, splits)) < self.min_samples_split:
                continue
            # information gain
            HxS = 0
            for split in splits:
                HxS += len(split) * self._entropy(split) / len(ids)
            gain = node.entropy - HxS 
            if gain < best_gain:
                continue
            best_gain = gain 
            best_splits = splits
            best_attribute = att
            order = values
        node.set_properties(best_attribute, order)
        child_nodes = [TreeNode(ids=split,
                       entropy=self._entropy(split), depth=node.depth+1) for split in best_splits]
        return child_nodes
    
    def _set_label(self, node):
        target_ids = self.target.iloc[node.ids]
        node.set_label(target_ids.mode().values[0])
    
    def predict(self, new_data):
        npoints = new_data.shape[0]
        labels = [None] * npoints
        for n in range(npoints):
            x = new_data.iloc[n, :]
            node = self.root
            while node.children:
                if x[node.split_attribute] in node.order:
                    child_index = node.order.index(x[node.split_attribute])
                    node = node.children[child_index]
                else:
                    break
            labels[n] = node.label
        return labels

# Usage
df = pd.read_csv('weather.csv')
X = df.iloc[:, :-1]  # All columns except the last one
y = df.iloc[:, -1]   # Last column (target)

tree = DecisionTreeID3(max_depth=3, min_samples_split=2)
tree.fit(X, y)
print(tree.predict(X))

['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']
